In [3]:
%load_ext autoreload
%autoreload 2

from schema2db import validate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np

In [12]:
inputfile = '../tests/testdata/testschema1.sql'
with open(inputfile, 'r') as f:
    inputstr = f.read()

In [14]:
components = inputstr.split(";")

In [15]:
components[0]

'create table prices (\n   itemid int(16) unsigned not null,\n   itemname varchar(20) not null,\n   price decimal(9,6) signed not null,\n   primary key (itemid)\n)'

In [16]:
table_str = components[0]

In [27]:
def get_table_name(words):
    if words[1].lower() != 'table'.lower():
        raise ValueError("You can only create tables, not {}".format(words[1]))
    else:
        return words[2]

def get_instruction(words):
    if words[0].lower() not in ['alter', 'create']:
        raise ValueError("{} operation not supported".format(words[0]))
    else:
        return words[0]


In [31]:
table_strs = table_str.split("\n")
table_words = [s.split() for s in table_strs]

instruction = get_instruction(table_words[0])
table_name = get_table_name(table_words[0])

table_words

prices create


[['create', 'table', 'prices', '('],
 ['itemid', 'int(16)', 'unsigned', 'not', 'null,'],
 ['itemname', 'varchar(20)', 'not', 'null,'],
 ['price', 'decimal(9,6)', 'signed', 'not', 'null,'],
 ['primary', 'key', '(itemid)'],
 [')']]

In [33]:
table_words[1][1]

'int(16)'